# Produce table-entry-page-retrieval-effectiveness

In [1]:
#!pip install trectools

from trectools import TrecQrel, TrecRun, TrecEval

In [2]:
QREL_DIR='../../Data/navigational-topics-and-qrels-ms-marco-v1/'
QREL_DIR_MARCO_V2='../../Data/navigational-topics-and-qrels-ms-marco-v2/'

RUN_DIR='/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/runs-ecir22/'
RUN_DIR_MARCO_V2='/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-probst/retrievalExperiments/runs-marco-v2-ecir22/'

TOPIC_TO_NAME={'entrypage-random': 'Random@V1', 'entrypage-popular': 'Popular@V1'}
QRELS={i: TrecQrel(QREL_DIR + 'qrels.msmarco-' + i + '.txt') for i in TOPIC_TO_NAME.keys()}
QRELS_MARCO_V2={i: TrecQrel(QREL_DIR_MARCO_V2 + 'qrels.msmarco-v2-' + i + '.txt') for i in TOPIC_TO_NAME.keys()}

In [13]:
APPROACH_TO_MARCO_V1_RUN_FILE={
    'BM25@2016-07': 'run.cc-16-07-anchortext.bm25-default.txt',
    'BM25@2017-04': 'run.cc-17-04-anchortext.bm25-default.txt',
    'BM25@2018-13': 'run.cc-18-13-anchortext.bm25-default.txt',
    'BM25@2019-47': 'run.cc-19-47-anchortext.bm25-default.txt',
    'BM25@2020-05': 'run.cc-20-05-anchortext.bm25-default.txt',
    'BM25@2021-04': 'run.cc-21-04-anchortext.bm25-default.txt',
    'BM25@16--21': 'run.cc-combined-anchortext.bm25-default.txt',
    'BM25@Content': 'run.ms-marco-content.bm25-default.txt',
    'BM25@Title': 'run.msmarco-document-v1-title-only.pos+docvectors+raw.bm25-default.txt',
    'BM25@Orcas': 'run.orcas.bm25-default.txt',
    'DeepCT@Anchor': 'run.ms-marco-deepct-v1-anserini-docs-cc-2019-47-sampled-test-overlap-removed-389979.bm25-default.txt',
    'DeepCT@Orcas': 'run.ms-marco-deepct-v1-anserini-docs-orcas-sampled-test-overlap-removed-390009.bm25-default.txt',
    'DeepCT@Train':'run.ms-marco-deepct-v1-anserini-docs-ms-marco-training-set-test-overlap-removed-389973.bm25-default.txt',
    'MonoT5': 'run.ms-marco-content.bm25-mono-t5-maxp.txt',
    'MonoBERT': 'run.ms-marco-content.bm25-mono-bert-maxp.txt',
    'LambdaMART@CTA':'run.ms-marco.lambda-mart-cta-trees-1000.txt',
    'LambdaMART@CTOA':'run.ms-marco.lambda-mart-ctoa-trees-1000.txt',
    'LambdaMART@CTO':'run.ms-marco.lambda-mart-cto-trees-1000.txt',
    'LambdaMART@CT':'run.ms-marco.lambda-mart-ct-trees-1000.txt',
}

APPROACH_TO_MARCO_V2_RUN_FILE={
    'BM25@Content': 'run.msmarco-doc-v2.bm25-default.txt',
    'BM25@Orcas': 'run.orcas-ms-marco-v2.bm25-default.txt',
    'BM25@2016-07': 'run.cc-16-07-anchortext.bm25-default.txt',
    'BM25@2017-04': 'run.cc-17-04-anchortext.bm25-default.txt',
    'BM25@2018-13': 'run.cc-18-13-anchortext.bm25-default.txt',
    'BM25@2019-47': 'run.cc-19-47-anchortext-v2.bm25-default.txt',
    'BM25@2020-05': 'run.cc-20-05-anchortext.bm25-default.txt',
    'BM25@2021-04': 'run.cc-21-04-anchortext.bm25-default.txt',
    'BM25@16--21': 'run.cc-union-16-to-21-anchortext-1000.bm25-default.txt',
    'DeepCT@Anchor': 'run.ms-marco-deepct-v2-anserini-docs-cc-2019-47-sampled-test-overlap-removed-389979.bm25-default.txt',
    'DeepCT@Orcas': 'run.ms-marco-deepct-v2-anserini-docs-orcas-sampled-test-overlap-removed-390009.bm25-default.txt',
    'DeepCT@Train':'run.ms-marco-deepct-v2-anserini-docs-ms-marco-training-set-test-overlap-removed-389973.bm25-default.txt',
    'MonoT5': 'run.ms-marco-content.bm25-mono-t5-maxp.txt',
    'MonoBERT': 'run.ms-marco-content.bm25-mono-bert-maxp.txt',
    'LambdaMART@CTA':'run.ms-marco.lambda-mart-cta-trees-1000.txt',
    'LambdaMART@CTOA':'run.ms-marco.lambda-mart-ctoa-trees-1000.txt',
    'LambdaMART@CTO':'run.ms-marco.lambda-mart-cto-trees-1000.txt',
    'LambdaMART@CT':'run.ms-marco.lambda-mart-ct-trees-1000.txt',
}

In [14]:
def recall(trec_eval, depth):
    import pandas as pd
    trecformat = trec_eval.run.run_data.sort_values(["query", "score", "docid"], ascending=[True,False,False]).reset_index()
    topX = trecformat.groupby("query")[["query","docid"]].head(depth)
    merged = pd.merge(topX[["query","docid"]], trec_eval.qrels.qrels_data[["query","docid","rel"]])

    nqueries = len(trec_eval.qrels.topics())
    result = merged[merged["rel"]>0].groupby("query")["rel"].count()
    
    return result.sum()/nqueries

def is_anchor_text(run_file):
    return '-anchortext' in run_file

def format_score(score, run_file, topics, position):
    # Manually maintained
    if is_anchor_text(run_file):
        max_scores = {
            'entrypage-random': [0.74, 0.82, 0.88, 0.66, 0.69, 0.8],
            'entrypage-popular': [0.62, 0.71, 0.85, 0.56, 0.63, 0.79],
        }
    else:
        max_scores =  {
            'entrypage-random': [0.59, 0.63, 0.69, 0.55, 0.58, 0.65],
            'entrypage-popular': [0.28, 0.32, 0.43, 0.27, 0.32, 0.43],
        }
        
    ret = '{:.2f}'.format(score)
    if max_scores[topics][position] <= score:
        return '{\\textbf{' + ret + '}} '
    else:
        return ret

def eval_on_marco_v1(run_file):
    ret = ' '
    for topics in ['entrypage-random', 'entrypage-popular']:
        run = TrecRun(RUN_DIR + topics + '/' + run_file)
        trec_eval=TrecEval(run, QRELS[topics])
        ret += '& ' + format_score(trec_eval.get_reciprocal_rank(), run_file, topics, 0)
        ret += '& ' + format_score(recall(trec_eval, 3), run_file, topics, 1)
        ret += '& ' + format_score(recall(trec_eval, 10), run_file, topics, 2)
    
    return ret + '&'

def eval_on_marco_v2(run_file):
    ret = ' '
    for topics in ['entrypage-random', 'entrypage-popular']:
        run = TrecRun(RUN_DIR_MARCO_V2 + topics + '/' + run_file)
        trec_eval=TrecEval(run, QRELS_MARCO_V2[topics])
        ret += (' ' if 'random' in topics else '& ') + format_score(trec_eval.get_reciprocal_rank(), run_file, topics, 3)
        ret += '& ' + format_score(recall(trec_eval, 3), run_file, topics, 4)
        ret += '& ' + format_score(recall(trec_eval, 10), run_file, topics, 5)
    
    return ret

def table_row(approach):
    v1_eval = ' & --- & --- & --- & --- & --- & --- &'
    
    if approach in APPROACH_TO_MARCO_V1_RUN_FILE:
        v1_eval = eval_on_marco_v1(APPROACH_TO_MARCO_V1_RUN_FILE[approach])
    
    v2_eval = ' --- & --- & --- & --- & --- & ---'
    
    if approach in APPROACH_TO_MARCO_V2_RUN_FILE:
        v2_eval = eval_on_marco_v2(APPROACH_TO_MARCO_V2_RUN_FILE[approach])
    
    return '& ' + approach + v1_eval + v2_eval + ' \\\\'


def table_entry_page_retrieval_effectiveness():
    return '''\\begin{table*}[bt]
    \\setlength{\\tabcolsep}{0.3em}
    \\caption{Overview of the retrieval effectiveness on 100~random entry page topics and 100~entry page topics for popular pages on version~1 of MS~Marco (V1) and version~2 of MS~Marco (V2). We Report the mean reciprocal rank (MRR) and the recall at~3 (R@3) and at~10 (R@10).}
    \\label{table-entry-page-retrieval-effectiveness}
    \\scriptsize
    \\begin{tabular*}{\\textwidth}{@{\\extracolsep{\\fill}}ll@{\\qquad}ccc@{\\quad}ccc@{\\quad}ccc@{\\quad}ccc@{}}

        \\toprule

        & & \\multicolumn{3}{@{}c@{\\quad}}{Random@V1} & \\multicolumn{3}{@{}c@{\\quad}}{Popular@V1} & \\multicolumn{3}{@{}c@{\\qquad}}{Random@V2} & \\multicolumn{3}{@{}c@{}}{Popular@V2} \\\\

        \\cmidrule(r{1em}){3-5} \\cmidrule(r{1em}){6-8} \\cmidrule(r{1em}){9-11} \\cmidrule{12-14}

        & & MRR & R@3 & R@10 & MRR & R@3 & R@10 & MRR & R@3 & R@10 & MRR & R@3 & R@10 \\\\

        \\midrule

        \\multirow{7}{*}{\\rotatebox[origin=c]{90}{\\parbox[c]{4em}{\\centering \\textbf{Anchor}}}}
        ''' + table_row('BM25@2016-07') + '''
        
        ''' + table_row('BM25@2017-04') + '''

        ''' + table_row('BM25@2018-13') + '''

        ''' + table_row('BM25@2019-47') + '''

        ''' + table_row('BM25@2020-05') + '''

        ''' + table_row('BM25@2021-04') + '''
        
        ''' + table_row('BM25@16--21') + '''

        \\midrule

        \\multirow{7}{*}{\\rotatebox[origin=c]{90}{\\parbox[c]{4em}{\\centering \\textbf{Baselines}}}}
        ''' + table_row('BM25@Content') + '''

        ''' + table_row('BM25@Orcas') + '''
        
        ''' + table_row('DeepCT@Anchor') + '''

        ''' + table_row('DeepCT@Orcas') + '''

        ''' + table_row('DeepCT@Train') + '''
        
        ''' + table_row('MonoT5') + '''
        
        ''' + table_row('MonoBERT') + '''
        
        ''' + table_row('LambdaMART@CTOA') + '''
        
        ''' + table_row('LambdaMART@CTO') + '''
        
        ''' + table_row('LambdaMART@CTA') + '''
        
        ''' + table_row('LambdaMART@CT') + '''
        
        \\bottomrule

    \\end{tabular*}
    \\vspace*{-2ex}
\\end{table*} 
'''

print(table_entry_page_retrieval_effectiveness())

\begin{table*}[bt]
    \setlength{\tabcolsep}{0.3em}
    \caption{Overview of the retrieval effectiveness on 100~random entry page topics and 100~entry page topics for popular pages on version~1 of MS~Marco (V1) and version~2 of MS~Marco (V2). We Report the mean reciprocal rank (MRR) and the recall at~3 (R@3) and at~10 (R@10).}
    \label{table-entry-page-retrieval-effectiveness}
    \scriptsize
    \begin{tabular*}{\textwidth}{@{\extracolsep{\fill}}ll@{\qquad}ccc@{\quad}ccc@{\quad}ccc@{\quad}ccc@{}}

        \toprule

        & & \multicolumn{3}{@{}c@{\quad}}{Random@V1} & \multicolumn{3}{@{}c@{\quad}}{Popular@V1} & \multicolumn{3}{@{}c@{\qquad}}{Random@V2} & \multicolumn{3}{@{}c@{}}{Popular@V2} \\

        \cmidrule(r{1em}){3-5} \cmidrule(r{1em}){6-8} \cmidrule(r{1em}){9-11} \cmidrule{12-14}

        & & MRR & R@3 & R@10 & MRR & R@3 & R@10 & MRR & R@3 & R@10 & MRR & R@3 & R@10 \\

        \midrule

        \multirow{7}{*}{\rotatebox[origin=c]{90}{\parbox[c]{4em}{\centering \textbf{Anc